# Simple Species Distribution Model in Python
## Introduction
Species distribution model are...

### Species Distribution Models 
Species Distribution Models (SDM)
We are going to produce an species distribution model using python and GBIF data.

### Species of interest
We selected a Magnolia species called Magnolia iltisiana. This is a Tree that inhabits in the western part of Mexico. 


## Methods
### Data download
For this guide we will use data downloaded from the Global Biodiversity Information Facility ([GBIF](https://www.gbif.org/)). the GBIF database contains a huge collection of information about all kinds of organisms. Our main interest in this database is the information about the distribution of the species.
For this guide we will search and download the information about our species Magnolia iltisiana.

![Main GBIF](/assets/images/assets/images/01_gbif_main.png "GBIF main site")

We have to select the correct species from the suggested results.

![GBIF search](/assets/images/assets/images/02_gbif_search.png "GBIF search results")

Here we can see all the data available for this species in the GBIF database. In our case our main interest are the ocurrences link above the images.

![GBIF species data](/assets/images/assets/images/03_gbif_M_iltisiana.png "GBIF species data")

In the occurrences site we found a table with all kind of information about the species. In particular we can find the localities were the species has been found. This data is gatered from different sources. We can review this in the Basis of record and Dataset columns.

![GBIF species occurrences](/assets/images/assets/images/04_gbif_occurrences.png "GBIF species occurrences")

Once we reviewed the data of our species we can download the database using the link at the top of the website. This will generate an unique download link with their corresponding reference. It is recommended to save this information for future reference.

As variables for our model we will use the 19 bioclimatic variables of [world clim](https://www.worldclim.org/data/worldclim21.html). We also going to include the elevation layer also from world clim. All layer were downloaded with a resolution of 30s and saved in a folder called *variables*.

![Worldclim variables](/assets/images/assets/images/05_worldclim.png "Worldclim variables")

#### Data preprocesing
We will mainly use the pandas and geopandas library for the data management and matplotlib.pyplot for the figures. For the clustering and predictive models we will use the modules of scikit learn. Aditionally we will use a couple of custom functions from the file sdm_functions.py


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import rasterio
from rasterio.plot import show
from shapely.geometry import Polygon
from sklearn import decomposition
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import sys

from sdm_functions import sdm_functions as fun


We start by reading the data downloaded from GBIF:

In [ ]:
sp = pd.read_csv('magnolia_iltisiana.txt', sep='\t')
sp

As we can see the original file includes several columns. For our case we only need the georeference data. In this case, not all datapoints include coordinates so we will discard the localities without this information. Optionally we can save this data as a new file for future references.

In [ ]:
sp = pd.read_csv('magnolia_iltisiana.txt', sep='\t', usecols=['scientificName', 'decimalLatitude', 'decimalLongitude']).dropna().drop_duplicates()
sp['presence'] = True
sp = sp.reset_index()
# sp.to_csv('magnolia_iltisiana_vars.csv', index=False) ### optional: saves data to a new csv file
sp

Now we have a dataframe with only 64 datapoints. We will convert dataframe into a geopandas dataframe and then we will extract the extent of the points to define the study area. We use a distance of 1 degree around the points to define a margin.
 visualize it in a map as follows:

In [ ]:
sp = gpd.GeoDataFrame(sp, crs="epsg:4326", geometry=gpd.points_from_xy(sp.decimalLongitude, sp.decimalLatitude))    
extent = fun.extent_poly(df=sp, margin_size=1, crs="epsg:4326")


Now we can visualize our localities in a map. We are importing the Mexican states as background of our map.

In [ ]:
dest = gpd.read_file("shapefiles/dest20gw.zip")
fig, ax = plt.subplots()
ax.set_aspect("equal")
plt.xlim([extent.bounds['minx'][0], extent.bounds['maxx'][0]])
plt.ylim([extent.bounds['miny'][0], extent.bounds['maxy'][0]])

dest.plot(ax=ax, color='lightgrey', edgecolor='darkgrey')
sp.plot(ax = ax, color="green", edgecolor = "black") 
plt.show()